In [2]:
from tkinter import *
from collections import deque

WIDTH = 900
HEIGHT = 600
POLE_WIDTH = 30
POLE_HEIGHT = 400
POLE1_BASE = (WIDTH/4,HEIGHT/2+POLE_HEIGHT/2)
POLE2_BASE = (WIDTH/2,HEIGHT/2+POLE_HEIGHT/2)
POLE3_BASE = (WIDTH*(3/4),HEIGHT/2+POLE_HEIGHT/2)
POLE_BASES = [POLE1_BASE,POLE2_BASE,POLE3_BASE]
TILE_WIDTH = 50
TILE_HEIGHT = 20
TILE_SCALING = 5
TILE_COLORS = ['#ffff00','#ff7700']
SELECTION_MAX = 100

class Hanoi:
    def __init__(self):
        self.window = Tk()
        self.window.title('Hanoi')
        self.canvas = Canvas(self.window, width=WIDTH, height=HEIGHT, bg="#36f5ce")
        self.canvas.pack()
        self.E1 = Entry(self.window, bg="#36f5ce", font="Courier")
        self.E1.bind('<Return>',self.draw)
        self.E1.pack(side=RIGHT)
        self.B1 = Button(self.window,command=self.solve,activebackground="#b8b8b8",bg="#4287f5",text="Solve")
        self.B1.pack(side=RIGHT)
        self.B2 = Button(self.window,command=self.window.destroy,text="Quit")
        self.B2.pack(side=BOTTOM)
        self.S1 = Scale(self.window,from_=1,to=1000,orient=HORIZONTAL,length=600, command=self.setDelay,label='Delay (ms)',
            resolution=1, bg="#36f5ce", font="Courier")
        self.S1.set(200)
        self.S1.pack(side=BOTTOM)
        self.delay = 200

        self.n = 0
        self.towers = None
        self.tiles = None
        self.ops = 0
        self.origin = 0
        self.target = 2

        self.initialize()

    def solve(self):
        self.n = self.handleEntry()
        self.resetTowers()
        self.draw(event=0)
        
        ai = HanoiAI(self)
        moves = ai.AIsolve()
        self.playSolution(moves, len(moves), 0)
        
    def playSolution(self, moves, n, i):
        if i < n:
            self.move(moves[i][0],moves[i][1])
            self.window.after(self.delay,self.playSolution,moves,n,i+1)

    def initialize(self):
        self.drawPoles()

    def draw(self, event=None):
        self.canvas.delete('all')
        self.drawPoles()
        self.drawTiles(event)
        


    def drawPoles(self):
        self.canvas.create_rectangle(WIDTH/4-POLE_WIDTH/2,HEIGHT/2-POLE_HEIGHT/2,WIDTH/4+POLE_WIDTH/2,HEIGHT/2+POLE_HEIGHT/2,fill="#5c2715",outline="black",width=2)
        self.canvas.create_rectangle(WIDTH/2-POLE_WIDTH/2,HEIGHT/2-POLE_HEIGHT/2,WIDTH/2+POLE_WIDTH/2,HEIGHT/2+POLE_HEIGHT/2,fill="#5c2715",outline="black",width=2)
        self.canvas.create_rectangle(WIDTH*(3/4)-POLE_WIDTH/2,HEIGHT/2-POLE_HEIGHT/2,WIDTH*(3/4)+POLE_WIDTH/2,HEIGHT/2+POLE_HEIGHT/2,fill="#5c2715",outline="black",width=2)


    def drawTiles(self, event=None):
        self.n = self.handleEntry()
        if event: 
            self.resetTowers()
        self.tiles = [None]*self.n
        for t, tower in enumerate(self.towers):
            t_height = len(tower)
            for i in range(t_height):
                self.tiles[tower[i]] = [POLE_BASES[t][0], POLE_BASES[t][1]-TILE_HEIGHT/2-(t_height-i-1)*TILE_HEIGHT]
                self.canvas.create_rectangle(self.tiles[tower[i]][0]-(tower[i]*TILE_SCALING+TILE_WIDTH/2), self.tiles[tower[i]][1]-TILE_HEIGHT/2, 
                    self.tiles[tower[i]][0]+(tower[i]*TILE_SCALING+TILE_WIDTH/2), self.tiles[tower[i]][1]+TILE_HEIGHT/2,
                    outline="#000",fill=TILE_COLORS[tower[i]%2], width=2)

    def handleEntry(self):
        input = self.E1.get()
        if input not in [str(i) for i in range(1,21)]:
            self.E1.delete(0,SELECTION_MAX)
            return 0
        self.E1.selection_range(0,SELECTION_MAX)
        return int(self.E1.get())

    def setDelay(self, delay):
        self.delay = delay

    def resetTowers(self):
        self.towers = [deque([i for i in range(self.n)]), deque(), deque()]

    def move(self,origin,target):
        self.ops += 1
        self.towers[target].appendleft(self.towers[origin].popleft())
        self.draw()

    def mainloop(self):
        self.window.mainloop()



class HanoiAI:
    def __init__(self, context: 'Hanoi'):
        self.ctx = context
        self.moves = []
        
    def AIsolve(self):
        self.rec(self.ctx.n, self.ctx.origin, self.ctx.target)
        return self.moves

    def rec(self, n, origin, target):
        if n == 0: return
        helper = 3 - origin - target
        self.rec(n-1, origin, helper)
        self.moves.append([origin,target])
        self.rec(n-1,helper,target)
        
        


hanoi = Hanoi()
hanoi.mainloop()


# tiles[i] = [x,y] of center of tile i, from 0 to n-1
# towers[i] = list of tiles in tower i, from top to bottom



Error: Canceled future for execute_request message before replies were done